In [1]:
import os
import sys

import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdummysubjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.ch

In [6]:
from tqdm import tqdm
from edc_constants.constants import NO
from edc_base.utils import age
from flourish_caregiver.models import CaregiverChildConsent
from flourish_child.models import ChildVisit, InfantHIVTesting9Months, \
    InfantHIVTestingAge6To8Weeks, InfantHIVTestingOther
from datetime import datetime
import pytz

import json


def read_json_file(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
    return data


infant_hiv_test_objs = read_json_file(
    '/Users/nimrodmunatsi/Downloads/infanthivtesting.json')

for infant_hiv_test_obj in tqdm(infant_hiv_test_objs):
    if infant_hiv_test_obj.get('fields').get('child_tested_for_hiv') == NO or infant_hiv_test_obj.get('fields').get('child_tested_for_hiv') == 'Dont_know':
        continue
    visit = infant_hiv_test_obj.get('fields').get('child_visit')
    subject_identifier = visit[0]
    child_consent = CaregiverChildConsent.objects.filter(
        subject_identifier=subject_identifier).latest('consent_datetime')
    if not (child_consent.child_dob and infant_hiv_test_obj.get('fields').get('child_test_date')):
        continue
        
    child_test_date = infant_hiv_test_obj.get('fields').get('child_test_date')  
    child_test_date = datetime.strptime(child_test_date, "%Y-%m-%d")
    report_datetime = infant_hiv_test_obj.get('fields').get('report_datetime')

    report_datetime = datetime.strptime(report_datetime, "%Y-%m-%dT%H:%M:%SZ")
    report_datetime = report_datetime.replace(tzinfo=pytz.UTC)
    age_at_test = age(child_consent.child_dob,child_test_date)
    
    try:
        child_visit = ChildVisit.objects.get(visit_code=visit[3],
                                             subject_identifier=subject_identifier)
    except ChildVisit.DoesNotExist:
        print(f'Child visit {visit[3]} does not exist for {subject_identifier}')
        continue

    else:

        options = {
            'child_visit': child_visit,
            'report_datetime': report_datetime,
            'child_tested_for_hiv': infant_hiv_test_obj.get('fields').get('child_test_date'),
            'child_test_date_estimated': infant_hiv_test_obj.get('fields').get('child_test_date_estimated'),
            'results_received': infant_hiv_test_obj.get('fields').get('results_received'),
            'recall_result_date': infant_hiv_test_obj.get('fields').get('recall_result_date'),
            'received_date': infant_hiv_test_obj.get('fields').get('received_date'),
            'result_date_estimated': infant_hiv_test_obj.get('fields').get('result_date_estimated'),
            'hiv_test_result': infant_hiv_test_obj.get('fields').get('hiv_test_result'),
            'additional_comments': infant_hiv_test_obj.get('fields').get('additional_comments'),
        }
        #test done at 6-8 weeks
        if age_at_test.years == 0 and age_at_test.months == 2:
            test_obj = InfantHIVTestingAge6To8Weeks.objects.create(
                **options
            )
        #test done at 9 months
        elif age_at_test.years == 0 and age_at_test.months == 9:
            test_obj = InfantHIVTesting9Months.objects.create(
                **options
            )
        #test done at 18 months
        elif age_at_test.years == 1 and age_at_test.months == 6:
            test_obj = InfantHIVTesting9Months.objects.create(
                **options
            )
        #test done at birth
        elif age_at_test.years == 0 and age_at_test.months == 0:
            test_obj = InfantHIVTesting9Months.objects.create(
                **options
            )
        else:
            test_obj = InfantHIVTestingOther.objects.create(
                **options
            )
        test_obj.save()


  0%|          | 1/265 [00:00<01:11,  3.67it/s]

Child visit 3001 does not exist for B142-040990558-4-60
Child visit 2007 does not exist for B142-040990608-7-10


  6%|▋         | 17/265 [00:11<02:11,  1.89it/s]

Child visit 2003 does not exist for B142-040991053-5-10


 11%|█         | 29/265 [00:21<02:57,  1.33it/s]

...

Child visit 2001 does not exist for B142-040991129-3-10


 62%|██████▏   | 163/265 [01:15<00:41,  2.45it/s]

Child visit 3001 does not exist for B142-040990479-3-10
Child visit 3001 does not exist for B142-040990563-4-10


 65%|██████▌   | 173/265 [01:16<00:40,  2.25it/s]


AgeValueError: Reference date 2022-01-30 00:00:00 None precedes DOB 2022-12-14 None. Got relativedelta(months=-10, days=-14, hours=-22)

In [14]:
dir(ChildVisit)

['ADMIN_SITE_NAME',
 'DoesNotExist',
 'Meta',
 'MultipleObjectsReturned',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_column_name_clashes',
 '_check_constraints',
 '_check_field_name_clashes',
 '_check_fields',
 '_check_id_field',
 '_check_index_together',
 '_check_indexes',
 '_check_local_fields',
 '_check_long_column_names',
 '_check_m2m_through_same_relationship',
 '_check_managers',
 '_check_model',
 '_check_model_name_db_lookup_clashes',
 '_check_ordering',
 '_check_property_name_related_field_accessor_clashes',
 '_check_single_primary_key',
 '_check_swappable',
 '_check_unique_together',
 '_check_visit_reason_keys',
 